In [22]:
import tweepy
import pymongo
import json

print("Dependencies imported")

Dependencies imported


In [20]:
#TO DO: Add to config file later

consumer_key = "mQXkO4dzpD3OPD1nOIT3fuD0j"
consumer_secret = "obt5r2qT1o878i5Eja7GYropJfCeUX76IXcjXRuHoJjIZnaard"
users_database_name = "users"
user_followers_database = "user_followers"
seed_user = "P9Ashwini"

seed_users = ["P9Ashwini"]

In [16]:
#Storing in MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")
users_db = client[users_database_name]
users_followers_db = client[users_followers_database]

In [18]:
collist = mydb.list_collection_names()
if users_database_name in collist:
    print("Users collection exists")

else:
    print("ERR: Users collection doesn't exist yet")

ERR: Users collection doesn't exist yet


In [19]:
#OAuth2 Authentication
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [24]:
# for tweet in tweepy.Cursor(api.search, q='tweepy').items(10):
#     print(tweet.text)
not_pretty = (api.get_user(seed_user))

TypeError: the JSON object must be str, bytes or bytearray, not User

In [42]:
#users object fields
for i in not_pretty._json.keys():
    print(i)
# print(not_pretty._json["following"])

id
id_str
name
screen_name
location
profile_location
description
url
entities
protected
followers_count
friends_count
listed_count
created_at
favourites_count
utc_offset
time_zone
geo_enabled
verified
statuses_count
lang
status
contributors_enabled
is_translator
is_translation_enabled
profile_background_color
profile_background_image_url
profile_background_image_url_https
profile_background_tile
profile_image_url
profile_image_url_https
profile_banner_url
profile_link_color
profile_sidebar_border_color
profile_sidebar_fill_color
profile_text_color
profile_use_background_image
has_extended_profile
default_profile
default_profile_image
following
follow_request_sent
notifications
translator_type
None
